In [1]:
import os
import re
import cv2
from PIL import Image
import numpy as np
import pandas as pd
import numpy.random as rnd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.utils as shuffle
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.regularizers  import l2
from keras.optimizers import Adam
from keras.models import Sequential, Model
from keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, merge, Lambda

Using TensorFlow backend.


In [2]:
def w_init (shape, dtype, name = None):
    values = rnd.normal(loc = 0, scale = 1e-2, size  = shape)
    return K.variable(values, name = name, dtype = dtype)

def b_init (shape, dtype, name = None):
    values = rnd.normal(loc = 0.5, scale = 1e-2, size = shape)
    return K.variable(values, name = name, dtype = dtype)

input_shape = (98, 98, 1)
left_input = Input(input_shape)
right_input = Input(input_shape)

In [7]:
siamnet = Sequential()
siamnet.add(Conv2D(64, (10, 10), activation = "relu", input_shape = input_shape))

siamnet.add(MaxPooling2D(pool_size=(2, 2)))

siamnet.add(Conv2D(128, (7, 7), activation ="relu"))

siamnet.add(MaxPooling2D(pool_size=(2, 2)))

siamnet.add(Conv2D(128, (4, 4), activation ="relu"))

siamnet.add(MaxPooling2D(pool_size=(2, 2)))

siamnet.add(Conv2D(256, (4, 4), activation ="relu"))

siamnet.add(Flatten())

siamnet.add(Dense(4096, activation = "sigmoid"))

In [10]:

encoded_left = siamnet(left_input)
encoded_right = siamnet(right_input)

L1_siamese = Lambda(lambda tensors:K.abs(tensors[0] - tensors[1]))
L1_dist = L1_siamese([encoded_left, encoded_right])

similarity = Dense(1, activation = "sigmoid")(L1_dist)

siamese_network = Model(inputs = [left_input, right_input], outputs = similarity)

siamese_network.compile(loss = "binary_crossentropy", optimizer = Adam(0.00006))

siamese_network.count_params()




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


27417409

In [11]:
size = 2
sample_size = 5000  

def prepare_data(size, sample_size):
    count = 0
    path = []
    image_pairs = []
    targets =  []
    
    dim1 = 98
    dim2 = 98
    img_pair = np.zeros([sample_size, 2, dim1, dim2])
    trgt = np.zeros([sample_size, 1])
    
    for fx in os.listdir('data'):
        filepath = 'data/' + fx
        path.append(fx)
        for j in range(int(sample_size/152)):
            index1 = 0
            index2 = 0
        
            while index1 == index2:
                index1 = np.random.randint(20)
                index2 = np.random.randint(20)
                
            image1 = cv2.imread(filepath + '/' + fx + '.' + str(index1 + 1) + '.jpg')
            image2 = cv2.imread(filepath + '/' + fx + '.' + str(index2 + 1) + '.jpg')
            
            gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
            gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
            
            gray1 = gray1[::size, ::size]
            gray2 = gray2[::size, ::size]
                        
            img_pair[count, 0, :, :] = gray1
            img_pair[count, 1, :, :] = gray2
            
            trgt[count] = 1

            count += 1
        
    count = 0

    imgImposite_pair = np.zeros([sample_size, 2, dim1, dim2])
    trgtImposite = np.zeros([sample_size, 1])
    
    for i in range(int(sample_size/20)):
        for j in range(20):
            index1 = 0
            index2 = 0
            
            while index1 == index2:
                index1 = np.random.randint(40)
                index2 = np.random.randint(40)
                
            image1 = cv2.imread('data/' + path[index1] + '/' +path[index1] + '.' + str(j+1) + '.jpg')
            image2 = cv2.imread('data/' + path[index2] + '/' +path[index2] + '.' + str(j+1) + '.jpg')
            
            gray1 = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
            gray2 = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
            
            gray1 = gray1[::size, ::size]
            gray2 = gray2[::size, ::size]
    
            imgImposite_pair[count, 0, :, :] = gray1
            imgImposite_pair[count, 1, :, :] = gray2
            
            trgtImposite[count] = 0
            
            count += 1

    image_pairs = np.concatenate([img_pair, imgImposite_pair], axis=0)/255
    targets = np.concatenate([trgt, trgtImposite], axis=0)
    
    return image_pairs, targets


In [12]:
image_pairs, targets = prepare_data(size, sample_size)

no_of_sample = image_pairs.shape[0]
pairs = image_pairs.shape[1]
img_rows = image_pairs[0][0].shape[0]
img_cols = image_pairs[0][0].shape[1]

Image_pairs = image_pairs.reshape(no_of_sample, pairs, img_rows, img_cols, 1)


In [15]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

tr_imgPairs, ts_imgPairs, tr_targets, ts_targets = train_test_split(Image_pairs, targets, 
                                                                    test_size = 0.25, 
                                                                    random_state = 0)

image1 = tr_imgPairs[:, 0]
image2 = tr_imgPairs[:, 1]

siamese_network.fit([image1, image2], tr_targets, validation_split=.25, batch_size=128, verbose=2, 
                    nb_epoch = 10)

pred = siamese_network.predict([ts_imgPairs[:, 0], ts_imgPairs[:, 1]])

def compute_accuracy(predictions, labels):
    return labels[predictions.ravel()] 

Instructions for updating:
non-resource variables are not supported in the long term


/Users/rishigarg/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  if sys.path[0] == '':


Train on 5625 samples, validate on 1875 samples
Epoch 1/10
 - 383s - loss: 0.5180 - val_loss: 0.4126
Epoch 2/10
 - 409s - loss: 0.3282 - val_loss: 0.2755
Epoch 3/10
 - 445s - loss: 0.2264 - val_loss: 0.2253
Epoch 4/10
 - 462s - loss: 0.1697 - val_loss: 0.1819
Epoch 5/10
 - 436s - loss: 0.1314 - val_loss: 0.1412
Epoch 6/10
 - 456s - loss: 0.0992 - val_loss: 0.1271
Epoch 7/10
 - 509s - loss: 0.0818 - val_loss: 0.1149
Epoch 8/10
 - 507s - loss: 0.0671 - val_loss: 0.1042
Epoch 9/10
 - 550s - loss: 0.0583 - val_loss: 0.0994
Epoch 10/10
 - 437s - loss: 0.0528 - val_loss: 0.0969


In [35]:
test_image1 = cv2.imread('9540636.8' + '.jpg')
test_image2 = cv2.imread('9540504.2' + '.jpg')

gray1 = cv2.cvtColor(test_image1, cv2.COLOR_BGR2GRAY)
gray2 = cv2.cvtColor(test_image2, cv2.COLOR_BGR2GRAY)

gray1 = gray1[::size, ::size]
gray2 = gray2[::size, ::size]

gray1 = gray1.reshape(1,98,98,1)
gray2 = gray2.reshape(1,98,98,1)

print(gray2.shape)

(1, 98, 98, 1)


In [36]:
pred = siamese_network.predict([gray1,gray2])
print(pred)

[[0.9775519]]


82.32


In [40]:
siamese_network.save("my_model.h5")

In [43]:
from keras.models import load_model
new_model = load_model("my_model.h5")
pred = new_model.predict([gray1,gray2])
print(pred)

[[0.9775519]]


In [45]:
score = new_model.evaluate([tr_imgPairs[:, 0], tr_imgPairs[:, 1]], tr_targets)

7500/7500 [==============================] - 143s 19ms/step


In [46]:
print(score)

0.057816594038407006


In [47]:
score = new_model.evaluate([ts_imgPairs[:, 0], ts_imgPairs[:, 1]], ts_targets)
print(score)

2500/2500 [==============================] - 49s 20ms/step
0.09393809925317764
